In [1]:
from dotenv import load_dotenv
import json
import os
import pandas as pd
import requests

In [2]:
load_dotenv()
EZRF_API_KEY = os.getenv('EZRF_API_KEY')
CHAIN_MAPPINGS = json.load(open('data/chains.json', 'r'))

In [3]:
def fetch_data(limit, cursor):
    url = f'https://ezrf-impact.vercel.app/api/trpc/projects.list?input=%7B%22json%22%3A%7B%22limit%22%3A{limit}%2C%22cursor%22%3A{cursor}%7D%7D'
    headers = {
        'content-type': 'application/json',
        'round-id': 'the-sunnys',
        'x-api-key': EZRF_API_KEY
    }

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        print(f"Data fetched successfully! (Page {cursor})")
        payload = response.json()
        json_data = payload['result']['data']['json']
        return json_data
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")

applications = []
est_apps = 2000
lim = 200
curs = 0
while curs * lim < est_apps:
    data = fetch_data(lim, curs)
    if data:
        applications.extend(data)
        curs += 1
    if not data or len(data) < lim:
        break
print(f"Total of {len(applications)} applications fetched.")

Data fetched successfully! (Page 0)
Data fetched successfully! (Page 1)
Data fetched successfully! (Page 2)
Data fetched successfully! (Page 3)
Data fetched successfully! (Page 4)
Data fetched successfully! (Page 5)
Data fetched successfully! (Page 6)
Data fetched successfully! (Page 7)
Total of 1421 applications fetched.


In [5]:
def clean_address(a):
    if not isinstance(a, str):
        return None
    a = a.lower().strip()
    if a[:2] != '0x':
        return None
    return a

chain_list = []
normalized_data = []
for (i,app) in enumerate(applications):
    
    profile = app.get('profile', {})
    if not profile:
        profile = {}
    profile_name = profile.get('name', '')
    metadata = app.get('metadata', {})
    awards = metadata.get('sunnyAwards', {})
    project_type = awards.get('projectType', '').title()
    if project_type == 'Other':
        project_type = 'Other Application'
    category = awards.get('category', '')
    if category == 'Other':
        category = 'Other Category'
    contracts = awards.get('contracts', [])    
    
    if len(contracts) > 1:
        print("WARNING: Array encountered at index:", i)
        break
    elif len(contracts) == 1:
        contract = contracts[0]
        address_type = 'contract'
        address = contract.get('address')
        chain_id = contract.get('chainId')
        chain_list.append(chain_id)
        chain = CHAIN_MAPPINGS.get(str(chain_id), 'All Superchain')
    else:
        address_type = 'mintingWallet'
        address = awards.get('mintingWalletAddress')
        chain_id = None
        chain = 'All Superchain'
    address = clean_address(address)
    if not address:
        address_type = 'N/A'
        chain = None

    app_data = {
        'id': app['id'],
        'uuid': app['uuid'],
        'attester': app['attester'],
        'recipient': app['recipient'],
        'time': app['time'],
        'name': app['name'],
        'schemaId': app['schemaId'],
        'status': app['status'],
        'round': app['round'],
        'profile_name': profile_name,
        'profile_url': f"https://warpcast.com/{profile_name}" if profile_name else '',
        'profile_image': profile.get('profileImageUrl', ''),
        'profile_banner': profile.get('bannerImageUrl', ''),
        'metadata_name': metadata.get('name', ''),
        'metadata_bio': metadata.get('bio', ''),
        'metadata_website': metadata.get('websiteUrl', ''),
        'project_type': project_type,
        'category': category,
        'category_details': awards.get('categoryDetails', ''),
        'avatar_url': awards.get('avatarUrl', ''),
        'cover_image_url': awards.get('coverImageUrl', ''),
        'address_type': address_type,
        'address': address,
        'chain_id': chain_id,
        'chain':  chain,
    }
    normalized_data.append(app_data)

df = pd.DataFrame(normalized_data)
df

,id,uuid,attester,recipient,time,name,schemaId,status,round,profile_name,...,metadata_website,project_type,category,category_details,avatar_url,cover_image_url,address_type,address,chain_id,chain
0,0x66076854e0f9ce49078c76ee39e2e9fae61a8526f406...,89eb0a1c-36f6-4455-af98-c822064425bb,0x8Bc704386DCE0C4f004194684AdC44Edf6e85f07,0xE4EE538019673501F4B75de5aF5CC073Ec0A1487,1724860607,0x,0xf8757b1e38ff1b0c1893e47f7d815367332bec28fea4...,approved,0x636d30617975733335303030357a7779623476747572...,ewokafloka,...,https://0x.org/,App,DEX,0x = developer focused APIs for building on De...,https://cdn.charmverse.io/user-content/0273f96...,https://cdn.charmverse.io/user-content/0273f96...,contract,0xdef1c0ded9bec7f1a1670819833240f027b25eff,8453.0,Base
1,0x5a587244ecf18246e881cda06bac1ece7c0995bb2faf...,403473a1-b2ad-4799-a7bb-834ca0b1f095,0x8Bc704386DCE0C4f004194684AdC44Edf6e85f07,0x0000000000000000000000000000000000000000,1724723573,1001 raisons,0xf8757b1e38ff1b0c1893e47f7d815367332bec28fea4...,approved,0x636d30617975733335303030357a7779623476747572...,kawz,...,https://www.sound.xyz/medmc/releases,Creator,Art NFTs,,https://cdn.charmverse.io/user-content/648db62...,https://cdn.charmverse.io/user-content/648db62...,mintingWallet,0x8f59aa0f586d6d941152b9075b344cdc42e9b024,NaN,All Superchain
2,0xc3d1a57ca75f8416819fd0e7158c85a224a30962ad20...,10a38b6d-e432-403b-bd9c-9e0bbbeef72b,0x8Bc704386DCE0C4f004194684AdC44Edf6e85f07,0x258a3790639F0dC736eAF5b8817706417C656a47,1725717115,1BITCUDA,0xf8757b1e38ff1b0c1893e47f7d815367332bec28fea4...,pending,0x636d30617975733335303030357a7779623476747572...,cudaoutofmemory,...,https://zora.co/@cudaoutofmemory?collection=zo...,Creator,Art NFTs,https://zora.co/@cudaoutofmemory?collection=zo...,https://cdn.charmverse.io/user-content/805b8ff...,https://cdn.charmverse.io/user-content/805b8ff...,mintingWallet,0xd0966b2d8d48e057105afa7a1b84bf1188fb00f0,NaN,All Superchain
3,0x9f6404eed4cb04fd2a0513887bb13c0ccc29dc9439cc...,8c12c1a5-d565-4851-abf2-1e22724816b0,0x8Bc704386DCE0C4f004194684AdC44Edf6e85f07,0x72bA95D9701B0aE496AB4320010a43a764905b1b,1724763747,1 Million Dream,0xf8757b1e38ff1b0c1893e47f7d815367332bec28fea4...,pending,0x636d30617975733335303030357a7779623476747572...,armin,...,https://zora.co/collect/zora:0x97c756b5509b50b...,Creator,Art NFTs,,https://cdn.charmverse.io/user-content/034b136...,https://cdn.charmverse.io/user-content/034b136...,mintingWallet,0x72ba95d9701b0ae496ab4320010a43a764905b1b,NaN,All Superchain
4,0x83343ef2cd78008735dd4f7d0de520b8678ee818fac3...,d8db2a21-5843-42a2-96af-378fba391603,0x8Bc704386DCE0C4f004194684AdC44Edf6e85f07,0xFA1aFC4534fc9F80a552e61Dd04CD8A172c821a6,1724762469,2021go in Web3,0xf8757b1e38ff1b0c1893e47f7d815367332bec28fea4...,pending,0x636d30617975733335303030357a7779623476747572...,jaxo,...,https://2021go.xyz,Other Application,Other Category,Web3 guide,https://cdn.charmverse.io/user-content/085011e...,,N/A,None,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1416,0x83f84ea97054ed128e47c90800be7e4d9a7f785b2a8f...,f46ec303-927a-450c-93ec-c8fc58d6f55a,0x8Bc704386DCE0C4f004194684AdC44Edf6e85f07,0x6B8288b5793D67eAFc89CA423F6A30b5ACe17dED,1725282971,Zuma,0xf8757b1e38ff1b0c1893e47f7d815367332bec28fea4...,pending,0x636d30617975733335303030357a7779623476747572...,zuma,...,https://zumaaaoptimis.com,Other Application,Staking,,,,N/A,None,NaN,None
1417,0xee53b36d13c1ad16a894d48230ac2923f77acb24e438...,195a757f-ed6d-4b60-8395-313e48b92997,0x8Bc704386DCE0C4f004194684AdC44Edf6e85f07,0xF14f94fF4c8c63Dfa73d82645Ef82743BB298231,1725505273,ZuztantivoZero,0xf8757b1e38ff1b0c1893e47f7d815367332bec28fea4...,pending,0x636d30617975733335303030357a7779623476747572...,moctezuma3ro,...,https://zuztantivozero.wtf,Other Application,Identity,Clothing & jewelry,https://cdn.charmverse.io/user-content/8e57f00...,https://cdn.charmverse.io/user-content/8e57f00...,N/A,None,NaN,None
1418,0x51d07a8792a25d8555f0d8145223911020ed99d9b1eb...,a6cd4000-7f17-48d5-a2e0-d0e0

In [6]:
# Flagging projects

# Flag 1: applied as 3 distinct projects from the same Farcaster account
project_count = df.groupby('profile_name')['uuid'].nunique()
flagged_farcaster_users = project_count[project_count > 3].index
df['flag_multiple_projects_same_profile'] = df['profile_name'].isin(flagged_farcaster_users)

# Flag 2: applied as an NFT creator category but no address
valid_categories_creator = ['Art NFTs', 'Other Media NFTs']
df['flag_creator_no_address'] = (df['category'].isin(valid_categories_creator)) & df['address'].isna()

# Flag 3: applied as an app but no address : chain mapping
other_categories_app = ['Channels', 'Frames', 'Other']
df['flag_app_missing_contract'] = (
    (~df['category'].isin(valid_categories_creator)) & 
    (~df['category'].isin(other_categories_app)) & 
    (df['address'].isna() | df['chain_id'].isna())
)

# Flag 4: applied as a channel but the url does not conform to the Warpcast channel pattern
df['flag_channel_no_channel'] = (
    (df['category'] == 'Channels')
    & (df['metadata_website'].str.contains("warpcast.com/~/channel/") == False)
)

# Flag 5: test project with Charmverse in the name :)
df['flag_charmverse_in_name'] = df['name'].str.contains('charmverse', case=False, na=False)

df['count_flags'] = df[[
    'flag_multiple_projects_same_profile', 
    'flag_creator_no_address', 
    'flag_app_missing_contract', 
    'flag_channel_no_channel',
    'flag_charmverse_in_name']].sum(axis=1)
df['has_flag'] = (df.count_flags > 0).astype(int)

In [7]:
with open("data/applications.json", "w") as f:
    json.dump(applications, f, indent=2)
    
df.drop(columns=[
    'attester', 'schemaId', 'round', 'profile_image', 'profile_banner',
    'metadata_bio', 'avatar_url', 'cover_image_url', 'category_details'
]).to_csv("data/applications_reviewed.csv")    